# LAB 11 - Random Forest for Regression

In this lab we will be extending the previous lab about Decision trees and build a Regression model using Random Forest.

For simplicity, we will be using the same dataset as the previous lab (you can find it in ECLASS).

**IMPORTANT:** For this lab, if you haven't finished your code from last week's lab on Decision trees, you will have the option to use the sklearn implementation for a regression tree. However, this doesn't mean that you should skip the previous lab. This is just so that you don't get behind with the content and you don't spend all your time today working on the previous lab. 

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

import pandas as pd
from sklearn.model_selection import train_test_split
import random

As mentioned before, use the Boston Housing data and prepare your train/val/test split as usual.

In [2]:
boston = pd.read_csv('BostonHousing.txt')
boston.head()

x, t = boston.values[:, :-1], boston.values[:, -1]

x_train, x_temp, t_train, t_temp = train_test_split(x, t, train_size = 0.7)
x_valid, x_test, t_valid, t_test = train_test_split(x_temp, t_temp, train_size = 0.5)

### Decision Tree Class:

In [3]:
class DTree():
    """ Decision Tree machine learning model 
    """
    def __init__(self) -> None:
        """Creating a Decision Tree
        """
        self.root = None
        
    def regression_criterion(self, region):
        """
        Implements the sum of squared error criterion in a region
        
        Parameters
        ----------
        region : ndarray
            Array of shape (N,) containing the target values for N datapoints in the training set.
        
        Returns
        -------
        sse : float
            The sum of squared error (= float("inf") for empty regions)
        """
        if len(region) == 0: return float("inf")

        y_hat = np.mean(region)
        sse = np.sum((region-y_hat)**2)
        return sse
    
    def split_region(self, region, feature_index, tau):
        """
        Given a region, splits it in two based on the passed feature, using tau as threshold
        
        Parameters
        ----------
        region : array of size (n_samples, n_features)
            a partition of the dataset (or the full dataset) to be split
        feature_index : int
            the index of the feature (column of the region array) used to make this partition
        tau : float
            The threshold used to make the partitions
            
        Return
        ------
        left_partition : array
            indices of the datapoints in `region` where feature < `tau`
        right_partition : array
            indices of the datapoints in `region` where feature >= `tau` 
        """
        mask = region[:, feature_index] < tau # here I create a mask that says True for a line where the "feature_index" column is lower than "tau"

        left_partition = np.nonzero(mask)[0]
        right_partition = np.nonzero(~mask)[0] # Using the logical inverse of the mask
        return left_partition, right_partition

    def get_split(self, X, y):
        """
        Given a dataset (full or partial), splits it on the feature of that minimizes the sum of squared error
        
        Parameters
        ----------
        X : array (n_samples, n_features)
            features 
        y : array (n_samples, )
            target labels
        
        Returns
        -------
        decision : dictionary
            keys:
            * 'feature_index' -> an integer that indicates the feature (column) of `X` on which the data is split
            * 'tau' -> the threshold used to make the split
            * 'left_region' -> array of indices where the `feature_index`th feature of X is lower than `tau`
            * 'right_region' -> indices not in `left_region`
        """
        best_error = float("inf")

        for col in range(X.shape[1]):
            for datapoint in range(X.shape[0]):
                tau = X[datapoint, col]
                l, r = self.split_region(X, col, tau)
                
                l_error = self.regression_criterion(y[l])
                r_error = self.regression_criterion(y[r])
                error = l_error + r_error

                if error < best_error:
                    best_error = error
                    decision = {"feature_index": col, "tau": tau, "left_region": l, "right_region": r}

        return decision

    def recursive_growth(self, min_samples, max_depth, error_criterion, X, y, node = None, current_depth = 0):
        """
        Recursively grows itself by optimizing the error in each node and obeying the passed stopping criterions.
        
        Parameters
        ----------
        min_samples : int
            parameter for stopping criterion if a node has <= min_samples datapoints
        max_depth : int
            parameter for stopping criterion if a node belongs to this depth
        error_criterion : float
            error value to act as stop criterion

        X : array (n_samples, n_features)
            features (full dataset)
        y : array (n_samples, )
            labels (full dataset)
        
        node : dictionary, defaults to self.root
            When passed, the recursive growth will start in this node, and not in the root.
            If the passed node is terminal, it contains only the "value" key, which determines the value to be used as a prediction.
            Else, the dictionary has the structure defined by `get_split`.
            
        current_depth : int, defaults to 0
            current distance from the root
        """
        if not node:
            self.root = self.get_split(X, y)
            node = self.root

        if "left_region" in node:
            l = node["left_region"]
            r = node["right_region"]
            
            # Process left
            if (y[l].size <= min_samples) or (current_depth == max_depth) or (self.regression_criterion(y[l]) <= error_criterion):
                node["left"] = {"value": np.mean(y[l])}
            else:
                node["left"] = self.get_split(X[l], y[l])
                self.recursive_growth(min_samples, max_depth, error_criterion, X, y, node["left"], current_depth+1)   
            
            # Process right
            if (y[r].size <= min_samples) or (current_depth == max_depth) or (self.regression_criterion(y[r]) <= error_criterion):
                node["right"] = {"value": np.mean(y[r])}
            else:
                node["right"] = self.get_split(X[r], y[r])
                self.recursive_growth(min_samples, max_depth, error_criterion, X, y, node["right"], current_depth+1)

    def print_tree(self, node = None, depth = 0):
        """Prints the whole tree

        :param node: Node from where to start printing, defaults to self.root
        :type node: dict, optional
        :param depth: Current depth of the node, defaults to 0 (root depth)
        :type depth: int, optional
        """
        if not node: 
            node = self.root

        if 'value' in node.keys():
            print('.  '*(depth-1), f"[{node['value']}]")
        else:
            print('.  '*depth, f'X_{node["feature_index"]} < {node["tau"]}')
            self.print_tree(node['left'], depth+1)
            self.print_tree(node['right'], depth+1)

    def __predict_sample(self, sample, node = None):
        """
        Makes a prediction based on the decision tree defined by `node`
        
        Parameters
        ----------
        sample : array of size (n_features,)
            a sample datapoint to be predicted
        node : dict, defaults to self.root
            A node created by one of the methods above
        """
        if not node: 
            node = self.root

        if "value" in node.keys():
            return node["value"]
        
        if sample[node["feature_index"]] < node["tau"]:
            return self.__predict_sample(sample, node["left"])
        
        return self.__predict_sample(sample, node["right"])
        
    def predict(self, X):
        """
        Makes a prediction based on the fitted training data
        
        Parameters
        ----------
        X : array of size (n_samples, n_features)
            n_samples predictions will be made

        Returns
        -------
        np.ndarray (n_samples, )
            Unidimensional array with predicted values or labels for each datapoint in the dataset 'X'
        """
        prediction = []
        for datapoint in X:
            prediction.append(self.__predict_sample(datapoint))
        
        return np.array(prediction)


## Exercise 1 -- Bootstrap

Also known as [bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating), this technique consists of making several samples with replacement of the original data, using each of the samples to train an estimator, and then aggregating the predictions using the average (this is also a type of model ensemble).

In [4]:
def bootstrap(X, num_bags=10):
    """
    Given a dataset and a number of bags,
    sample the dataset with replacement.
    
    This function does not return a copy
    of the datapoints, but a list of indices
    with compatible dimensionality
    
    Parameters
    ----------
    X : ndarray
        A dataset
    num_bags : int, default 10
        The number of bags to create
    
    Returns
    -------
    list of ndarray
        The list contains `num_bags` integer one-dimensional ndarrays.
        Each of these contains the indices corresponding to the 
        sampled datapoints in `X`
    
    Notes
    -----
    * The number of datapoints in each bach will
      match the number of datapoints in the given
      dataset.
    """
    rng = np.random.default_rng(0) # you can change the seed, or use 0 to replicate my results
    return [rng.integers(0, X.shape[0], X.shape[0]) for i in range(num_bags)] 
    # Your code here

In [5]:
rng = np.random.default_rng(0)
X_small = rng.random(size=(100,2))
bags = bootstrap(X_small)
bags[0]

array([85, 63, 51, 26, 30,  4,  7,  1, 17, 81, 64, 91, 50, 60, 97, 72, 63,
       54, 55, 93, 27, 81, 67,  0, 39, 85, 55,  3, 76, 72, 84, 17,  8, 86,
        2, 54,  8, 29, 48, 42, 40,  2,  0, 12,  0, 67, 52, 64, 25, 61, 76,
       38, 46, 99, 80, 98, 37, 68, 95, 65, 84, 68, 70, 38, 87, 13, 57, 72,
       84, 52, 37, 31, 42, 48, 71, 88,  7, 93, 53, 35, 67, 57, 25, 32, 71,
       59, 50, 33, 76, 39, 32, 89, 26, 22, 71, 62,  4,  8, 37, 83],
      dtype=int64)

## Exercise 2 -- Aggregation

The second part of bagging.

In [6]:
def aggregate_regression(preds):
    """
    Aggregate predictions by several estimators
    
    Parameters
    ----------
    preds : list of ndarray
        Predictions from multiple estimators.
        All ndarrays in this list should have the same
        dimensionality.
        
    Return
    ------
    ndarray
        The mean of the predictions
    """
    return np.mean(np.array(preds), 0)
    # Your code here

## Exercise 3 -- Random Forest for regression

Using the functions you implemented above, it is now time to put all of them together to train several decision trees and then ensemble them to output a single prediction. For the random forest, however, we need to select a subset of features at each split on the decision tree. 

For this part, you can use the sklearn implementation of Random forest for regression as your estimator for each set of features and bags. See below an example of how to do this, and always remember to check the necessary documentation when using an external function.

Some parameters you will have to set are: 
* num_features: number of features per estimator
* min_samples: min number of samples per leaf node
* max_depth: maximum depth of the decision tree (each estimator)
* num_estimators: number of decision trees you will create using each bag and random set of features

In [7]:
def get_features_set(feat_num, chosen_num):
    """Chooses a subset of features between the total number

    :param feat_num: Original number of features
    :type feat_num: int
    :param chosen_num: Number of features to be chosen between the original ones
    :type chosen_num: int

    :return: List of indexes to access a subset of the features
    :rtype: list
    """
    chosen_feats = random.choices(range(feat_num), k = chosen_num)
    return chosen_feats

def fit_bagging(X, y, num_features, num_estimators, min_samples, max_depht):
    """Creates a random forest model with 'num_estimators' trees, each trained with a subset of 'num_features' features between the original ones in the dataset 'X'

    :param X: Full dataset to fit in the random forest's trees
    :type X: np.ndarray
    :param y: Full dataset labels
    :type y: np.ndarray

    :param num_features: Number of features to use in each singular tree
    :type num_features: int
    :param num_estimators: Number of trees in the random forest
    :type num_estimators: int
    :param min_samples: Parameter for a tree's stopping criterion if a node has <= min_samples datapoints
    :type min_samples: int
    :param max_depht: Parameter for a tree's stopping criterion if a node belongs to this depth
    :type max_depht: int

    :return: Returns a list containing 'num_estimators' objects from the DTree() class, trained through bagging process
    :rtype: list
    """
    n_features = X.shape[1]
    bags = bootstrap(X, num_estimators)
    forest = []

    for i in range(num_estimators):
        estimator = DTree()

        # Separating a subset with fewer features and the same amount of datapoints (random with replacement)
        features = get_features_set(n_features, num_features)
        X_subset = X[bags[i]][:, features]
        y_subset = y[bags[i]]
        
        # Training the tree with the defined subsets of data
        estimator.recursive_growth(min_samples, max_depht, 0, X_subset, y_subset)

        forest.append(estimator)
        
    return forest

def predict(estimators: list[DTree], X):
    """Makes the prediction for a Random Forest Model

    :param estimators: List with many DTree class estimators
    :type estimators: list[DTree]
    :param X: Dataset to predict the labels
    :type X: np.ndarray
    :return: Returns an array with the average from all the predictions in the random forest
    :rtype: np.ndarray
    """
    predictions = []

    for tree in estimators:
        predictions.append(tree.predict(X))
    
    return aggregate_regression(predictions)

def RMSE(predict, target):
    return np.sqrt(np.mean((predict - target)**2))


In [9]:
## your code goes here:
num_features = 4 # The slides say the common value is approx sqrt(n_features)
num_estimators = 50
min_samples = 25
max_depht = 5

random_forest = fit_bagging(x_train, t_train, num_features, num_estimators, min_samples, max_depht)
y_pred = predict(random_forest, x_valid)

print("Validation RMSE =", RMSE(y_pred, t_valid))

Validation RMSE = 8.80501069845815
